In [ ]:
# Importar las librerías necesarias
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, f1_score, precision_score, recall_score
import matplotlib.pyplot as plt
import numpy as np
import itertools
from tensorflow.keras import backend as K
from sklearn.metrics import precision_recall_fscore_support

# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Directorios de entrenamiento y validación
train_dir = '/content/drive/MyDrive/Dataset_entrenamiento'
val_dir = '/content/drive/MyDrive/Dataset_validacion'

# Parámetros
img_height, img_width = 224, 224  # Tamaño de las imágenes
batch_size = 32  # Tamaño de lotes para el entrenamiento
epochs = 25  # Número de épocas para el entrenamiento

# Data Augmentation para entrenamiento
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Data Augmentation para validación (solo se reescala)
validation_datagen = ImageDataGenerator(rescale=1./255)

# Cargar los datos de entrenamiento y validación
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'  # Clasificación binaria: carcinoma vs sano
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

# Cargar el modelo DenseNet121 preentrenado (ImageNet)
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Congelar las capas del modelo preentrenado para no entrenarlas
base_model.trainable = False

# Crear el modelo completo
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),  # Evitar sobreajuste
    Dense(1, activation='sigmoid')  # Salida para clasificación binaria
])

# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

# Evaluar el modelo en el conjunto de validación
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f'Exactitud en validación: {val_accuracy * 100:.2f}%')

# Predicciones
validation_generator.reset()  # Reiniciar el generador
predictions = model.predict(validation_generator)
predictions = np.round(predictions).astype(int)  # Redondear las predicciones a 0 o 1

# Extraer las etiquetas reales
true_labels = validation_generator.classes

# Matriz de confusión
conf_matrix = confusion_matrix(true_labels, predictions)
print('Matriz de confusión:')
print(conf_matrix)

# Calcular métricas (Precision, Recall, F1 Score)
precision = precision_score(true_labels, predictions)
recall = recall_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions)
print(f'Precisión: {precision * 100:.2f}%')
print(f'Recall (Sensibilidad): {recall * 100:.2f}%')
print(f'F1 Score: {f1:.2f}')

# Curva ROC y AUC
fpr, tpr, thresholds = roc_curve(true_labels, predictions)
roc_auc = auc(fpr, tpr)
print(f'AUC: {roc_auc:.2f}')

# Visualizar la curva ROC
plt.figure()
plt.plot(fpr, tpr, label=f'Curva ROC (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos (FPR)')
plt.ylabel('Tasa de Verdaderos Positivos (TPR)')
plt.title('Curva ROC')
plt.legend(loc='lower right')
plt.show()

# Calcular el Dice Coefficient
def dice_coefficient(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1)

dice_coeff = dice_coefficient(true_labels, predictions)
print(f'Dice Coefficient: {K.eval(dice_coeff):.2f}')
